In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from datetime import timedelta, date
import time
import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [4]:
def sent_a_arxiv_mail(cat, you='mniu0726@gmail.com'):
    web_url = 'https://arxiv.org/list/' + cat + '/pastweek?show=500'
    page = requests.get(web_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    info_raw = soup.find_all('dl')[0]
    info_url = info_raw.find_all('dt')
    instant_raw = info_raw.find_all('dd')

    paper_name = []
    authors = []
    urls = []
    for i in range(len(instant_raw)):
        paper_name.append(instant_raw[i].text.split('\n')[3].replace('Title: ', '')) 
        authors.append(' '.join(instant_raw[i].text.split('\n')[6:][:instant_raw[i].text.split('\n')[6:].index("")]))
        urls.append("https://arxiv.org" + str(info_url[i].find_all(title='Download PDF')[0]['href']) + ".pdf")
    # print(paper_name)
    # print()
    origin_str = '<p>'
    for i in range(len(instant_raw)):
        new_str = '<strong><a href=\"' + urls[i] + '\">' + paper_name[i] + '</a></strong><br />' + authors[i] + '<br /><br/>'
        origin_str = origin_str + new_str
    origin_str = origin_str + '</p>'
    origin_str = origin_str.encode('utf-8')
    
    me = "mniu0726@gmail.com"
    
    msg = MIMEMultipart('alternative')
    if cat == 'cs.AI':
        msg['Subject'] = "New Papers from arxiv -- Artificial Intelligence"    
    elif cat == 'cs.DS':
        msg['Subject'] = "New Papers from arxiv -- Data Structures and Algorithms"
    elif cat == 'cs.DB':
        msg['Subject'] = "New Papers from arxiv -- Databases"
    elif cat == 'cs.NA':
        msg['Subject'] = "New Papers from arxiv -- Numerical Analysis"
    elif cat == 'stat.ML':
        msg['Subject'] = "New Papers from arxiv -- Machine Learning"
    elif cat == 'q-fin':
        msg['Subject'] = "New Papers from arxiv -- Quantitative Finance"
    elif cat == 'cs.CE':
        msg['Subject'] = "New Papers from arxiv -- Computational Engineering, Finance, and Science"
    else:
        msg['Subject'] = "New Papers from arxiv -- unknown category"
            
    msg['From'] = me
    msg['To'] = you

    # <h3> is the bold trigger, 3 is the font size, the bigger the smaller
    html = origin_str

    part2 = MIMEText(html, 'html')
    msg.attach(part2)
    mail = smtplib.SMTP('smtp.gmail.com', 587)
    mail.ehlo()
    mail.starttls()

    mail.login('mniu0726@gmail.com', 'nfbcjnm19910726')
    mail.sendmail(me, you, msg.as_string())
    mail.quit()

In [6]:
Mu_list = ['cs.AI', 'cs.DB', 'stat.ML', 'cs.DS']
yun_list = ['q-fin', 'cs.CE', 'stat.ML']

for i in range(len(Mu_list)):
    sent_a_arxiv_mail(Mu_list[i], 'mniu19910726@gmail.com')
# for i in range(len(yun_list)):
#     sent_a_arxiv_mail(yun_list[i], you='qingyun1993@gmail.com')